In [1]:
import re
import numpy as np
import sys
import zipfile
import pandas as pd


In [2]:
path = r'Talk Radio.zip'

In [3]:
movie = []

In [4]:
print("opening file")
archive = zipfile.ZipFile(path, 'r')
files = zipfile.ZipFile.namelist(archive)
for file in files:
	filename = re.findall(r'.*\.srt', file)
	if len(filename)>0:
		filename = filename[0]
		break

opening file


In [213]:
movie_data = archive.read(filename)
movie_data = movie_data.decode().split('\r\n')

In [214]:
print("Processing file")
df = pd.DataFrame(movie_data, columns=['dialog'])
print(df.head())
print(df.columns)

Processing file
                          dialog
0                             ﻿1
1  00:00:26,635 --> 00:00:29,763
2        There's a tornado watch
3   for a portion of West Texas.
4                               
Index(['dialog'], dtype='object')


In [237]:
def cleansing(df, n, x):
    def remove_blank_lines(df):
        df.replace("", np.nan, inplace=True)
        df = df.dropna()
        return df
    
    def remove_brackets(df):
        df = df.replace(r'^-?\(.*\)$', '', regex=True)
        return df
    
    def remove_specialchars(df):
        df = df.replace(r'[\=\-\{\}\[\]\^\#]', '', regex=True)
        return df
    
    def _minus(df):
        return df

    def remove_nonchar_lines(df):
        df = df.drop(df[df.dialog.str.contains(r'^\d?\d?\d?\d|\d\d:\d\d:\d\d,.*', regex=True)].index)
        return df
    
    def remove_first_n_lines(df, n):
        df = df.iloc[n:]
        return df
    
    
    df = remove_blank_lines(df)
    df = remove_brackets(df)
    df = remove_nonchar_lines(df)
    df = remove_specialchars(df)
    df = remove_blank_lines(df)
    df = remove_first_n_lines(df, n)
    df = remove_blank_lines(df)
    
    return df

In [238]:
df_movie = cleansing(df, 1, 0)

In [239]:
df_movie

,dialog
2,There's a tornado watch
3,for a portion of West Texas.
7,Brian Chadwick is with the county
8,sheriff's office in Sweetwater.
12,We're trying to get the word out to
...,...
8818,Now he's dead.
8822,I don't know if you
8823,"understand that analogy,"
8827,but it's the clearest


In [252]:
movie_txt = df_movie.dialog.values.tolist()

In [253]:
sent = ''
new_l = []
for t in movie_txt:
    if t[-1] not in ['.','?','!']:
        sent+=t+' '
    else:
        sent+=t
        new_l.append(sent)
        sent = ''
        
df_movie_updated = pd.DataFrame(columns=['dialog'])
df_movie_updated['dialog'] = new_l
df_movie_updated.head()

,dialog
0,There's a tornado watch for a portion of West ...
1,Brian Chadwick is with the county sheriff's of...
2,We're trying to get the word out to the commun...
3,We hope this thing's not gonna develop into a ...
4,"but after Saragosa, we're not taking any chances."


In [256]:
print("converting to csv")
df_movie_updated.to_csv('talk_radio_conv.csv')

converting to csv
